# Reinforcement Learning With Practice

I'm writing this notebook for a better understanding of Reinforcement Learning and its practical implementation. I got the motive while reading [A Survey on Deep Reinforcement Learning Algorithms for Robotic Manipulation](https://www.mdpi.com/1424-8220/23/7/3762). I found it obscure to apply the algorithms to real-world problems, like I don't know what's the reward or policy.

I introduce all the algorithms briefly and use them to solve the [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) environment from OpenAI Gym.

---
**Notations**

A Markov devision process (MDP) is defined as a tuple $\langle S, A, r, T, \gamma \rangle$ where:
- $S$ - current state
- $A$ - action
- $r$ - reward for taking action $A$ in state $S$
- $T$ - state transition function
- $\gamma$ - discount factor implying that a reward obtained in the future is worth a smaller amount than an immediate reward
---

## About CartPole
1. action space: a ndarray with shape `(1,)` which can take values `{0, 1}` indicating the direction of the fixed force the cart is pushed with.
2. observation space: a ndarray with shape `(4,)` containing the cart position, cart velocity, pole angle, and pole velocity at the tip.
3. reward: `1` for every step taken, including the termination step.
4. termination: the episode ends when the pole is more than 15 degrees from vertical or the cart moves more than 2.4 units from the center.
5. truncation: the episode ends after 500 steps for v1 (200 for v0)

## 1. Value-Based RL

### 1.1 Q-Learning
$$ Q(s, a) = r(s, a) + \gamma \max_{a} Q(s', a) \tag{1} $$

$Q(s, a)$ is the Bellman action-value function, which estimates how good it is to take an action at a given state. Q-Learning is off-policy that learns the optimal policy directly.

### 1.2 SARSA
$$ Q(s, a) = Q(s, a) + \alpha \left[ R + \gamma Q(s', a') - Q(s, a) \right] \tag{2}$$
SARSA is on-policy, which means $a'$ in $Q(s', a')$ follows the current policy.

### 1.3 Deep Q-Learning (DQN)
Deep Q-learning (DQN) was introduced by Mnih et al. [18], which uses a neural network to approximate the Q-values. The update rule depends on the values produced by the network itself, making convergence diffucult. To address this, the DQN algorithm introduces the use of a replay buffer and target network $\theta'$. The replay buffer stores past interactions as a list of tuples, which can be sampled to update the value and policy networks. This allows the network to learn from individual tuples multiple times and reduces dependence on the current experience. The target networks are time-delayed copies of the policy and Q-networks, and their parameters are updated according to the following equations:

$$ \theta_Q' \leftarrow \tau \theta_Q + (1 - \tau) \theta_Q' \tag{4}$$
$$ \theta_\mu' \leftarrow \tau \theta_\mu + (1 - \tau) \theta_\mu' \tag{5}$$

where $\theta_\mu'$ and $\theta_Q'$ denote the parameters of the policy and Q-networks, respectively. The loss function such as the MSE loss can be:
$$ L = \left(Q(s, a;\theta) - (r + \gamma \max_{a'} Q(s', a';\theta')) \right) ^ 2 \tag{3}$$

<center>
    <img src="./images/dqn.webp" alt="example">
</center>


### 1.4 Double Deep Q-Learning (Double DQN)
$$ Q(s,a;\theta) = r + \gamma Q(s', argmax_{a'} Q(s', a'; \theta); \theta') \tag{6}$$
The main neural network, $\theta$ determines the best next action $a'$, while the target network is used to evaluate this action and compute its Q-value. This simple change has been shown to reduce overestimations of Q-values in DQN.


Watch [Double Deep Q-Learning](double_dqn.ipynb) for code example.

### 1.5 Dueling Deep Q-Learning
Dueling DQN improves upon traditional DQN by decomposing the Q-values into two separate components
- the value function $V(s)$, which the expected reward for a given state
- the advantage function $A(s, a)$, which reflects the relative advantage of taking a particular action compared to other actions

$$ Q(s,a) = V(s) + (A(s, a) - \frac{1}{|A|} \sum_{a'} A(s, a')) \tag{7}$$

<center>
    <img src="./images/dueling_dqn.webp" alt="example">
</center>


## 2. Policy-Based RL
Policy gradient (PG) methods are widely used reinforcement learning algorithms that are particularly well-suited to situations with continuous action spaces. The goal of an RL agent using a PG method is to maximize the expected reward, $J(\pi_\theta)=E_{\tau \sim \pi_\theta}[R(\tau)]$, by adjusting the policy parameters $\theta$. A standard approach to finding the optimal policy is to use gradient ascent, in which the policy parameters are updated according to the following rule:
$$ \theta_{t+1} = \theta_t + \alpha \nabla J(\pi_{\theta_t}) \tag{8}$$

This gradient can be further expanded and reformulated as:
$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla \theta \log \pi_\theta(a_t|s_t) R(\tau) \right] \tag{9}$$

In PG methods, the policy function, which maps states to actions, is learned explicitly and actions are selected without using a value function.

### 2.1 Vanilla Policy Gradient (VPG)
In RL, it is often more important to understand the relative advantage of a particular action, rather than its absolute effectiveness.
$$ A_\pi(s, a) = Q_\pi(s, a) - V_\pi(s) \tag{10}$$
- $A_\pi(s, a)$ - advantage function
- $Q_\pi(s, a)$ - action-value function
- $V_\pi(s)$ - state-value function for the policy $\pi$
- $\tau$ - trajectory according to the policy $\pi_\theta$

$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^T \nabla \theta \log \pi_\theta(a_t|s_t) A_\pi(s, a) \right] \tag{11}$$

<center>
    <img src="./images/vpg.webp" alt="example">
</center>

### 2.2 Trust Region Policy Optimization (TRPO)
Trust region policy optimization (TRPO) constrains the optimization of a policy to a trust region. This region is defined as the area in which local approximations of the function are accurate, and the maximum step size is determined within it. The trust region is then iteratively expanded or shrunk based on how well the new approximation performs.

The policy update in TRPO is given by the following optimization problem, which uses the Kullback–Leibler (KL) divergence between the old and new policies as a measure of change:
$$ \nabla J(\pi_\theta) = E_{\tau \sim \pi_\theta} \left[ \frac{\pi \theta(a_t|st)}{\pi \theta_{old}(a_t|st)} \hat{A}t \right] \tag{12}$$
$$ subject to E_{\tau \sim \pi_\theta} \left[ KL[\pi_{\theta_{old}}(\cdot|s_t), \pi_\theta(\cdot|s_t)] \right] \le \delta \tag{13}$$

$\delta$ is the size of the trust region, and the KL divergence between the old and new policies must be less than $\delta$. This optimization problem can be solved using the conjugate gradient method.

### 2.3 Proximal Policy Optimization (PPO)

Proximal policy optimization (PPO) is an algorithm that aims to address the overhead issue of TRPO by incorporating the constraint into the objective function as a penalty:
$$ E_{\tau \sim \pi_\theta} \left[ \frac{\pi \theta(a_t|st)}{\pi \theta_{old}(a_t|st)} \hat{A}t \right] - C \cdot \overline{KL}\pi \theta_{old} (\pi \theta) \tag{14}$$

One challenge of PPO is choosing the appropriate value for C. To address this, the algorithm updates C based on the magnitude of the KL divergence. If the KL divergence is too high, C is increased, and, if it is too low, C is decreased. This allows for effective optimization over the course of training.